# 自動化幾何圖形生成 (Reflection 模式)

此 Notebook 示範如何使用 Reflection 模式：
1. Writer 階段：提取條件、補全參數、生成 Matplotlib 腳本
2. Reflector 階段：對生成的腳本進行反思與優化建議
3. 執行並展示圖形結果

In [1]:
import os
import json
import io
from groq import Groq
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Circle
from datetime import datetime
from dotenv import load_dotenv

# 設定環境變數
load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# 日誌資料夾
LOG_ROOT = "log"
os.makedirs(LOG_ROOT, exist_ok=True)

# 1. 提取約束 (Writer)

In [2]:

def extract_constraints(text: str) -> str:
    system = (
        "你是一個幾何題目標註助理，將題目轉為 JSON 約束，請直接輸出 JSON 格式，不需要額外文字"
    )
    return client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role":"system","content":system},
            {"role":"user","content":text}
        ]
    ).choices[0].message.content



# 2. 補全參數 (Writer)

In [3]:
def supplement_parameters(constraints_json: str) -> str:
    system = "根據 Constraints JSON，產生點座標與結構的純 JSON，請直接輸出 JSON 格式，不需要額外文字"
    return client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role":"system","content":system},
            {"role":"user","content":constraints_json}
        ]
    ).choices[0].message.content

# 3. 生成 Matplotlib 腳本 (Writer)

In [4]:

def generate_plt_script(params_json: str) -> str:
    system = (
        "基於參數 JSON，撰寫 Matplotlib 繪圖程式，並儲存至 ./log/plot.png，請直接以純文字輸出不需要 ````python` 或 `#` 開頭的程式碼，以便後續直接進行執行。"    
    )
    user = f"以下是參數：{params_json}"
    return client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role":"system","content":system},
            {"role":"user","content":user}
        ]
    ).choices[0].message.content

# 4. 反思並優化腳本 (Reflector)

In [5]:

def reflect_script(script: str) -> str:
    system = (
        "你是一個程式碼審閱者，請檢查以下 Matplotlib 腳本，指出優化建議。"
    )
    user = f"請審閱並優化此程式：\n{script}"
    return client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role":"system","content":system},
            {"role":"user","content":user}
        ]
    ).choices[0].message.content

# 5. 執行並紀錄函式

In [6]:

def exec_and_log(script: str):
    scope = {'plt': plt, 'Polygon': Polygon, 'Circle': Circle, 'io': io, 'params':{}}
    buf = io.BytesIO()
    scope['buf'] = buf
    exec(script, scope)
    buf.seek(0)
    from IPython.display import Image, display
    display(Image(data=buf.read()))

## 使用流程示例

In [8]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 自動化幾何圖形生成（Reflection 模式）")

    # 輸入題目
    question = gr.Textbox(label="題目敘述（國中程度）", lines=2, placeholder="例如：畫一個平行四邊形 ABCD，頂點A(0,0), B(2,0), C(3,1), D(1,1)")

    # 各階段輸出區
    constraints_out = gr.Textbox(label="1. Constraints JSON", interactive=False)
    params_out      = gr.Textbox(label="2. Parameters JSON", interactive=False)
    script_out      = gr.Code(label="3. Generated Matplotlib 腳本", language="python")
    reflection_out  = gr.Textbox(label="4. Reflection 建議", interactive=False)

    # 一鍵執行完整流程
    def full_pipeline(text):
        # Writer 階段
        c = extract_constraints(text)
        p = supplement_parameters(c)
        s = generate_plt_script(p)
        # Reflector 階段
        r = reflect_script(s)
        # 執行並回傳圖片路徑
        img_path = exec_and_log(s)
        return c, p, s, r, img_path

    run_btn = gr.Button("執行完整流程")
    run_btn.click(
        fn=full_pipeline,
        inputs=question,
        outputs=[constraints_out, params_out, script_out, reflection_out]
    )

    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://8355d2f313b3108b95.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


c:\Users\ryan\anaconda3\Lib\site-packages\gradio\blocks.py:1829: UserWarning: A function (full_pipeline) returned too many output values (needed: 4, returned: 5). Ignoring extra values.
    Output components:
        [textbox, textbox, code, textbox]
    Output values returned:
        ["```json
{
  "type": "parallelogram",
  "vertices": [
    {
      "name": "A",
      "x": 0,
      "y": 0
    },
    {
      "name": "B",
      "x": 2,
      "y": 0
    },
    {
      "name": "C",
      "x": 3,
      "y": 1
    },
    {
      "name": "D",
      "x": 1,
      "y": 1
    }
  ]
}
```", "```json
{
  "points": [
    {
      "x": 0,
      "y": 0
    },
    {
      "x": 2,
      "y": 0
    },
    {
      "x": 3,
      "y": 1
    },
    {
      "x": 1,
      "y": 1
    }
  ],
  "structure": {
    "type": "parallelogram",
    "connections": [
      {
        "from": 0,
        "to": 1
      },
      {
        "from": 1,
        "to": 2
      },
      {
        "from": 2,
        "to": 3
      },

c:\Users\ryan\anaconda3\Lib\site-packages\gradio\blocks.py:1829: UserWarning: A function (full_pipeline) returned too many output values (needed: 4, returned: 5). Ignoring extra values.
    Output components:
        [textbox, textbox, code, textbox]
    Output values returned:
        ["```json
{
  "figure": "平行四邊形",
  "vertices": ["A", "B", "C", "D"],
  "edges": [
    {"start": "A", "end": "B"},
    {"start": "B", "end": "C"},
    {"start": "C", "end": "D"},
    {"start": "D", "end": "A"}
  ],
  "properties": {
    "opposite_sides_parallel": true,
    "opposite_sides_equal": true,
    "opposite_angles_equal": true
  }
}
```", "```json
{
  "points": {
    "A": [0, 0],
    "B": [2, 0],
    "C": [3, 2],
    "D": [1, 2]
  },
  "structure": {
    "type": "平行四邊形",
    "vertices": ["A", "B", "C", "D"],
    "edges": [
      {"start": "A", "end": "B"},
      {"start": "B", "end": "C"},
      {"start": "C", "end": "D"},
      {"start": "D", "end": "A"}
    ]
  }
}
```", "import matplotlib.pypl

In [10]:
import matplotlib.pyplot as plt
import json

parameters = {
  "points": {
    "A": [0, 0],
    "B": [2, 0],
    "C": [3, 2],
    "D": [1, 2]
  },
  "structure": {
    "type": "平行四邊形",
    "vertices": ["A", "B", "C", "D"],
    "edges": [
      {"start": "A", "end": "B"},
      {"start": "B", "end": "C"},
      {"start": "C", "end": "D"},
      {"start": "D", "end": "A"}
    ]
  }
}

points = parameters["points"]
structure = parameters["structure"]
vertices = structure["vertices"]
edges = structure["edges"]

x = []
y = []
for vertex in vertices:
    x.append(points[vertex][0])
    y.append(points[vertex][1])

x.append(x[0])
y.append(y[0])

plt.figure(figsize=(6,6))
plt.plot(x, y, '-o')

for i, vertex in enumerate(vertices):
    plt.annotate(vertex, (x[i], y[i]))

plt.title(structure["type"])
plt.gca().set_aspect('equal')
plt.savefig('./log/plot.png')
plt.show()

C:\Users\ryan\AppData\Local\Temp\ipykernel_70736\2876778881.py:45: UserWarning: Glyph 24179 (\N{CJK UNIFIED IDEOGRAPH-5E73}) missing from font(s) DejaVu Sans.
  plt.savefig('./log/plot.png')
C:\Users\ryan\AppData\Local\Temp\ipykernel_70736\2876778881.py:45: UserWarning: Glyph 34892 (\N{CJK UNIFIED IDEOGRAPH-884C}) missing from font(s) DejaVu Sans.
  plt.savefig('./log/plot.png')
C:\Users\ryan\AppData\Local\Temp\ipykernel_70736\2876778881.py:45: UserWarning: Glyph 22235 (\N{CJK UNIFIED IDEOGRAPH-56DB}) missing from font(s) DejaVu Sans.
  plt.savefig('./log/plot.png')
C:\Users\ryan\AppData\Local\Temp\ipykernel_70736\2876778881.py:45: UserWarning: Glyph 37002 (\N{CJK UNIFIED IDEOGRAPH-908A}) missing from font(s) DejaVu Sans.
  plt.savefig('./log/plot.png')
C:\Users\ryan\AppData\Local\Temp\ipykernel_70736\2876778881.py:45: UserWarning: Glyph 24418 (\N{CJK UNIFIED IDEOGRAPH-5F62}) missing from font(s) DejaVu Sans.
  plt.savefig('./log/plot.png')


<Figure size 600x600 with 1 Axes>